In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

2024-04-01 00:28:20.706735: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
ls

data/  model1/  model4/  requirements.txt*


In [5]:
cos_df = pd.read_csv('data/Cosmetic_products_sales.csv')
# cos_df = pd.read_csv('data/cos_sales.csv')
cos_df

,ParentSKU,Site Id,Year,Month,Category Name ID,Unit,Qty,Price,Net Sales calculated,Cash Discount,Amount to Customer,MRP,Pack Size,Pack Unit Id,State,Zone,Master Category,Size,Rank,Date
0,F0033,AHMEDABAD,2017,4,0,NO,1621.0,54,82557,20,93640,80.0,30.0,GM,DL,WEST2,3,30gm,4,2017-04-01
1,F0033,AHMEDABAD,2017,5,0,NO,651.0,51,31473,21,35760,80.0,30.0,GM,DL,WEST2,3,30gm,4,2017-05-01
2,F0033,AHMEDABAD,2017,6,0,NO,457.0,46,19580,12,22293,70.0,30.0,GM,DL,WEST2,3,30gm,4,2017-06-01
3,F0033,AHMEDABAD,2017,7,0,NO,1985.0,41,78144,0,78144,70.0,30.0,GM,DL,WEST2,3,30gm,4,2017-07-01
4,F0033,AHMEDABAD,2017,8,0,NO,6.0,38,228,0,228,70.0,30.0,GM,DL,WEST2,3,30gm,4,2017-08-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34945,T0270,VIJAYAWADA,2019,9,1,NO,1008.0,40,38321,0,38321,60.0,120.0,GM,DL,SOUTH,5,120gm,44,2019-09-01
34946,T0270,VIJAYAWADA,2019,10,1,NO,1179.0,40,44215,0,44215,60.0,120.0,GM,DL,SOUTH,5,120gm,44,2019-10-01
34947,T0270,VIJAYAWADA,2019,12,1,NO,81.0,40,2947,0,2947,60.0,120.0,GM,DL,SOUTH,5,120gm,44,2019-12-01
34948,T0270,VIJAYAWADA,2020,1,1,NO,580.0,40,21125,0,21125,60.0,120.0,GM,DL,SOUTH,5,120gm,44,2020-01-01


In [6]:
cos_df = cos_df.drop(columns=['Category Name ID', 'Unit', 'Pack Unit Id','Master Category', 'Rank', 'Date','Size']).dropna()
cos_df

,ParentSKU,Site Id,Year,Month,Qty,Price,Net Sales calculated,Cash Discount,Amount to Customer,MRP,Pack Size,State,Zone
0,F0033,AHMEDABAD,2017,4,1621.0,54,82557,20,93640,80.0,30.0,DL,WEST2
1,F0033,AHMEDABAD,2017,5,651.0,51,31473,21,35760,80.0,30.0,DL,WEST2
2,F0033,AHMEDABAD,2017,6,457.0,46,19580,12,22293,70.0,30.0,DL,WEST2
3,F0033,AHMEDABAD,2017,7,1985.0,41,78144,0,78144,70.0,30.0,DL,WEST2
4,F0033,AHMEDABAD,2017,8,6.0,38,228,0,228,70.0,30.0,DL,WEST2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34945,T0270,VIJAYAWADA,2019,9,1008.0,40,38321,0,38321,60.0,120.0,DL,SOUTH
34946,T0270,VIJAYAWADA,2019,10,1179.0,40,44215,0,44215,60.0,120.0,DL,SOUTH
34947,T0270,VIJAYAWADA,2019,12,81.0,40,2947,0,2947,60.0,120.0,DL,SOUTH
34948,T0270,VIJAYAWADA,2020,1,580.0,40,21125,0,21125,60.0,120.0,DL,SOUTH


In [7]:
string_encoded = ['ParentSKU', 'Site Id','State','Zone']

for i in string_encoded:
    cos_df[i] = pd.factorize(cos_df[i])[0]



In [8]:
def clean_shampoo_data(shampoo_df,window_size=5):
    shampoo_np = shampoo_df['Net Sales calculated'].to_numpy()
        
    x = []
    y = []
    for i in range(len(shampoo_np)-window_size):
        row = [[j] for j in shampoo_np[i:i+window_size].astype(float)]    
        x.append(row)
        y.append(shampoo_np[i+window_size].astype(float))
    
    
    return np.array(x), np.array(y)
shampoo_x, shampoo_y = clean_shampoo_data(cos_df,5)
shampoo_x.shape, shampoo_y.shape


((34945, 5, 1), (34945,))

In [9]:
import pandas as pd


default = [4,2017]
current = default.copy()

missing_times = []
last = ''

counter = 0
# Iterate through each row in the dataframe
for index, row in cos_df.iterrows():
    curr = row['Site Id']
    month = row['Month']
    year = row['Year']

    # print(curr,last)
    counter += 1
    if curr != last:
        current = default.copy()
        last = curr
    # print(curr,last)
    # print(current, month, year)


    while month != current[0] or year != current[1]:
        dup = row.copy()
        dup['Month'] = current[0]
        dup['Year'] = current[1]
        
        missing_times.append(dup.tolist())
        current[0] += 1
        if current[0] > 12:
            current[0] = 1
            current[1] += 1
    current[0] += 1
    if current[0] > 12:
        current[0] = 1
        current[1] += 1

missing_df = pd.DataFrame(missing_times, columns=cos_df.columns)
cos_df = pd.concat([cos_df, missing_df], ignore_index=True)

# Sort the DataFrame by ParentSKU, Year, and Month
cos_df = cos_df.sort_values(by=['ParentSKU','Site Id','Year', 'Month']).reset_index(drop=True)

print(missing_times)



[[0.0, 0.0, 2017.0, 10.0, 1522.0, 46.0, 64639.0, 0.0, 64639.0, 70.0, 30.0, 0.0, 0.0], [0.0, 2.0, 2017.0, 10.0, 540.0, 46.0, 23019.0, 0.0, 23019.0, 70.0, 30.0, 0.0, 5.0], [0.0, 4.0, 2017.0, 10.0, 1429.0, 45.0, 60205.0, 0.0, 60205.0, 70.0, 30.0, 0.0, 6.0], [0.0, 4.0, 2017.0, 11.0, 1429.0, 45.0, 60205.0, 0.0, 60205.0, 70.0, 30.0, 0.0, 6.0], [0.0, 8.0, 2017.0, 4.0, 429.0, 47.0, 18467.0, 3.0, 19727.0, 70.0, 30.0, 0.0, 5.0], [0.0, 8.0, 2017.0, 8.0, 143.0, 42.0, 5559.0, 0.0, 5559.0, 70.0, 30.0, 0.0, 5.0], [0.0, 9.0, 2017.0, 4.0, 1536.0, 39.0, 60134.0, 0.0, 60134.0, 70.0, 30.0, 0.0, 13.0], [0.0, 9.0, 2017.0, 5.0, 1536.0, 39.0, 60134.0, 0.0, 60134.0, 70.0, 30.0, 0.0, 13.0], [0.0, 9.0, 2017.0, 6.0, 1536.0, 39.0, 60134.0, 0.0, 60134.0, 70.0, 30.0, 0.0, 13.0], [0.0, 9.0, 2017.0, 7.0, 1536.0, 39.0, 60134.0, 0.0, 60134.0, 70.0, 30.0, 0.0, 13.0], [0.0, 9.0, 2017.0, 8.0, 1536.0, 39.0, 60134.0, 0.0, 60134.0, 70.0, 30.0, 0.0, 13.0], [0.0, 9.0, 2017.0, 9.0, 1536.0, 39.0, 60134.0, 0.0, 60134.0, 70.0, 30.0

In [10]:
from sklearn.preprocessing import MinMaxScaler

limit = 30000
scalar = MinMaxScaler()


original_shape = shampoo_x.shape
shampoo_x = shampoo_x.reshape(-1, original_shape[-1])
shampoo_x = scalar.fit_transform(shampoo_x)
shampoo_x = shampoo_x.reshape(original_shape)

shampoo_train_x, shampoo_train_y = shampoo_x[:limit],shampoo_y[:limit]
shampoo_test_x, shampoo_test_y = shampoo_x[limit:],shampoo_y[limit:]


model = Sequential()
model.add(LSTM(units=400, input_shape=(shampoo_train_x.shape[1], shampoo_train_x.shape[2])))
model.add(Dense(32, 'relu'))
model.add(Dense(1, 'linear'))
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])
# Train the model

2024-04-01 00:28:27.130724: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-01 00:28:27.212542: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-01 00:28:27.212591: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-01 00:28:27.215117: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-01 00:28:27.215190: I external/local_xla/xla/stream_executor

In [ ]:
# model.fit(shampoo_train_x, shampoo_train_y, epochs=10, batch_size=32)

Epoch 1/10


938/938 ━━━━━━━━━━━━━━━━━━━━ 133s 141ms/step - loss: 20202385408.0000 - root_mean_squared_error: 142051.4375
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 234s 249ms/step - loss: 18254731264.0000 - root_mean_squared_error: 134845.6875
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 68s 72ms/step - loss: 16392129536.0000 - root_mean_squared_error: 127911.7969
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 14s 15ms/step - loss: 14581465088.0000 - root_mean_squared_error: 120621.6250
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - loss: 16258349056.0000 - root_mean_squared_error: 127387.9766
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 15s 16ms/step - loss: 15980796928.0000 - root_mean_squared_error: 126274.9688
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 16s 16ms/step - loss: 15049186304.0000 - root_mean_squared_error: 122403.6484
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 14s 15ms/step - loss: 13803857920.0000 - root_mean_squared_error: 117245.1641
Epoch 9/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - loss: 1

In [ ]:
train_predictions = model.predict(shampoo_test_x).flatten()
train_results = pd.DataFrame(data={'Train Predictions':train_predictions, 'Actuals':shampoo_test_y})
train_results

NameError: name 'model' is not defined

In [ ]:
cos_df

,ParentSKU,Site Id,Year,Month,Qty,Price,Net Sales calculated,Cash Discount,Amount to Customer,MRP,Pack Size,State,Zone
0,0.0,0.0,2017.0,4.0,1621.0,54.0,82557.0,20.0,93640.0,80.0,30.0,0.0,0.0
1,0.0,0.0,2017.0,5.0,651.0,51.0,31473.0,21.0,35760.0,80.0,30.0,0.0,0.0
2,0.0,0.0,2017.0,6.0,457.0,46.0,19580.0,12.0,22293.0,70.0,30.0,0.0,0.0
3,0.0,0.0,2017.0,7.0,1985.0,41.0,78144.0,0.0,78144.0,70.0,30.0,0.0,0.0
4,0.0,0.0,2017.0,8.0,6.0,38.0,228.0,0.0,228.0,70.0,30.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42206,51.0,24.0,2019.0,10.0,1179.0,40.0,44215.0,0.0,44215.0,60.0,120.0,0.0,3.0
42207,51.0,24.0,2019.0,11.0,81.0,40.0,2947.0,0.0,2947.0,60.0,120.0,0.0,3.0
42208,51.0,24.0,2019.0,12.0,81.0,40.0,2947.0,0.0,2947.0,60.0,120.0,0.0,3.0
42209,51.0,24.0,2020.0,1.0,580.0,40.0,21125.0,0.0,21125.0,60.0,120.0,0.0,3.0


In [ ]:
def clean_cos_data(cos_df,window_size=6):
    cos_np = cos_df.astype(float).to_numpy()   
    x = []
    y = []
    for i in range(len(cos_np)-window_size):
        row = [j for j in cos_np[i:i+window_size]]    
        x.append(row)
        y.append(cos_np[i+window_size][6].astype(float))
    return np.array(x), np.array(y)
cos_x, cos_y = clean_cos_data(cos_df,6)
cos_x.shape, cos_y.shape


((42205, 6, 13), (42205,))

In [ ]:
limit = 30000

cos_train_x, cos_train_y = cos_x[:limit],cos_y[:limit]
cos_test_x, cos_test_y = cos_x[limit:],cos_y[limit:]

temp_training_mean = np.mean(cos_train_x[:, :, 0])
temp_training_std = np.std(cos_train_x[:, :, 0])
                           
def preprocess(X):
  X[:, :, 0] = (X[:, :, 0] - temp_training_mean) / temp_training_std
  return X

cos_train_x = preprocess(cos_train_x)
cos_test_x = preprocess(cos_test_x)
cos_test_x = preprocess(cos_test_y)
cos_test_x = preprocess(cos_test_y)
cos_train_x


IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed

In [ ]:
model4 = Sequential()
model4.add(InputLayer((6, 13)))
model4.add(LSTM(64))
model4.add(Dense(124, 'relu'))
model4.add(Dense(1, 'linear'))

model4.summary()

NameError: name 'Sequential' is not defined

In [ ]:
cp4 = ModelCheckpoint('model4/model4.keras', save_best_only=True)
model4.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model4.fit(cos_train_x, cos_train_y, validation_data=(cos_test_x, cos_test_y), epochs=100, callbacks=[cp4])

NameError: name 'ModelCheckpoint' is not defined

In [ ]:
train_predictions = model.predict(cos_test_x).flatten()
train_results = pd.DataFrame(data={'Train Predictions':train_predictions, 'Actuals':cos_test_y})
train_results

 45/382 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

382/382 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


,Train Predictions,Actuals
0,2.922769,10138.0
1,3.071050,7010.0
2,3.638699,18249.0
3,3130.087402,18249.0
4,3264.268066,18249.0
...,...,...
12200,8483.885742,44215.0
12201,6704.796387,2947.0
12202,1701.919189,2947.0
12203,4.481054,21125.0
